### アダマールテストと位相推定

さて、一通り準備がととのったので、固有値を求める量子アルゴリズムについて解説しよう。まず、アダマールテストとよばれる以下のような量子回路（figure01.pdf）を考える。真ん中にある制御ユニタリ演算$\Lambda(U)$は、上の量子ビットが$|0\rangle$の場合にはなにもせず、$|1\rangle$の場合には$U$を作用させるユニタリ演算である。

$$
\Lambda (U) = |0\rangle \langle 0| \otimes I + |1\rangle \langle 1|  \otimes U
$$

つまり、１つ目の量子ビットが$|0\rangle$か$|1\rangle$かによって条件分岐して「なにもしない」、「$U$を作用させる」という演算が実行される。従来のコンピュータでは条件分岐は同時に実行することができないが、量子コンピュータでは重ね合わせとして、同時並列的に実行することができる。

さて、アダマールテストの説明に戻ろう。最初は簡単のために量子状態$|\psi \rangle$を
ユニタリー演算（行列）$U$の固有値$e^{i \lambda}$の固有状態（固有ベクトル）とする：
\begin{eqnarray}
U|\psi \rangle = e^{i \lambda} |\psi\rangle.
\end{eqnarray}

ステップ1までの計算で
\begin{eqnarray}
\frac{1}{\sqrt{2}} (|0\rangle  + |1\rangle) \otimes |\psi  \rangle 
\end{eqnarray}
が得られる。
ステップ２で制御$U$演算を作用させることによって、
**固有値が１つめの量子ビットの相対位相として得られる**（これは位相キックバックとしばしば呼ばれている）:
\begin{eqnarray}
&&\frac{1}{\sqrt{2}} (|0\rangle \otimes |\psi  \rangle  
+ |1\rangle \otimes U|\psi \rangle )
\\
&=&\frac{1}{\sqrt{2}} (|0\rangle \otimes |\psi  \rangle  
+e^{i \lambda} |1\rangle \otimes |\psi \rangle )
\\
&=&
\frac{1}{\sqrt{2}} (|0\rangle   
+e^{i \lambda} |1\rangle )\otimes |\psi  \rangle.
\end{eqnarray}
最後のステップ３のアダマール演算によって
\begin{eqnarray}
\left(\frac{1+e^{i\lambda}}{2}|0\rangle  
+\frac{1-e^{i\lambda}}{2} |1\rangle \right)\otimes |\psi  \rangle 
\label{eq01}
\end{eqnarray}
が得られる。
一つ目の量子ビットを測定すると測定結果$m=0,1$を得る確率は
\begin{eqnarray}
p_{m}=\left|\frac{1+(-1)^m e^{i\lambda}}{2}\right|^2 =\frac{1+(-1)^m \cos \lambda}{2}
\end{eqnarray}
となる
$|\psi \rangle$、$U$、$\langle \psi |$は
それぞれ$2^n$次元の列ベクトル、$2^n \times 2^n$行列、
$2^n$次元の行ベクトルなので、
このアダマールテストを古典コンピュータ上で愚直に計算すると
指数的に大きなメモリーの確保と演算回数が必要になる。
一方で、量子コンピューターでは、
確率分布$p_m$のもとで$m$がサンプルされる。
$\cos \lambda$を
ある誤差$\epsilon$で推定したい場合は、
その逆数$1/\epsilon$の多項式回程度サンプルすればよいことになる。

さらに、固有値が$\pm 1$の場合にはこのアダマールテストによって、固有ベクトルを得ることができる。たとえば、$U=H$の場合を考えてみよう。補助量子ビットを$|0\rangle$、アダマールテストの入力$|\psi\rangle$も$|0\rangle$とする。

In [13]:
from IPython.display import Image, display_png
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit,QubitBra
init_printing() # ベクトルや行列を綺麗に表示するため
from sympy.physics.quantum.gate import X,Y,Z,H,S,T,CNOT,SWAP,CPHASE,CGateS
from IPython.display import Image, display_png
from sympy import *


state = Qubit('00')

制御H演算は、`CGateS()`を用いて

In [3]:
ctrlH = CGateS(1,H(0))
represent(ctrlH,nqubits=2)

⎡1  0  0    0  ⎤
⎢              ⎥
⎢0  1  0    0  ⎥
⎢              ⎥
⎢      √2   1  ⎥
⎢0  0  ──   ── ⎥
⎢      2    √2 ⎥
⎢              ⎥
⎢      1   -√2 ⎥
⎢0  0  ──  ────⎥
⎣      √2   2  ⎦

測定前の状態は、

In [4]:
H(1)*ctrlH*H(1)*state

H ⋅C ⎛H ⎞⋅H ⋅❘00⟩
 1  1⎝ 0⎠  1     

In [5]:
qapply(H(1)*ctrlH*H(1)*state)

√2⋅❘00⟩   ❘00⟩   √2⋅❘01⟩   √2⋅❘10⟩   ❘10⟩   √2⋅❘11⟩
─────── + ──── + ─────── - ─────── + ──── - ───────
   4       2        4         4       2        4   

測定は、測定用の関数が用意されており、１つめの量子ビットの測定は`measure_partial`を用いて得ることができる。`measure_partial`では測定後の状態と測定の確率がリストとして出力される。１つめの量子ビットが0だった場合の量子状態は[0][0]要素を参照すればよいので、

In [6]:
from sympy.physics.quantum.qubit import measure_all, measure_partial, measure_all_oneshot, measure_partial_oneshot
measured_state_zero = measure_partial(qapply(H(1)*ctrlH*H(1)*state),(1))[0][0]
simplify(measured_state_zero)

(√2 + 2)⋅❘00⟩ + √2⋅❘01⟩
───────────────────────
          ________     
      2⋅╲╱ √2 + 2      

これがHの固有ベクトルであることは、２つめの量子ビットにHを作用させると（sumpyのインデックスは１つめが1、２つめが0になるよう対応させていることに注意）、

In [7]:
simplify(qapply(H(0)*measured_state_zero))

√2⋅❘00⟩ + 2⋅❘00⟩ + √2⋅❘01⟩
──────────────────────────
           ________       
       2⋅╲╱ √2 + 2        

となりもとに戻る。固有状態であることがわかった。同様に1の測定結果を得た場合は、固有値-1の固有状態であることも確認できるので試してもらいたい。

In [8]:
measured_state_one = measure_partial(qapply(H(1)*ctrlH*H(1)*state),(1,))[1][0]
simplify(qapply(H(0)*measured_state_one))

-2⋅❘10⟩ + √2⋅❘10⟩ + √2⋅❘11⟩
───────────────────────────
           _________       
       2⋅╲╱ -√2 + 2        

固有値が$\pm 1$ではない場合も、アダマールテストの出力を入力として繰り返すと$U$の固有状態に状態が収束していくので、興味がある人は試してもらいたい。

同じ計算を、必ずしも固有ベクトルとは限らない、一般の入力に対して行うと、測定前の状態は、
$$
 |0\rangle \frac{I+U}{2} |\psi \rangle  +  |1\rangle  \frac{I-U}{2} |\psi \rangle 
$$
となり、0もしくは1が得られる確率は、
$$
p_0 = \frac{1+ {\rm Re} \langle \psi | U | \psi \rangle }{2}
\\
p_1 = \frac{1- {\rm Re} \langle \psi | U | \psi \rangle }{2}
$$
となる。つまり、ベクトル$|\psi \rangle$でユニタリ行列Uを挟んだ値をアダマールテストのサンプル結果の推定から
計算することができる。もちろん、量子ビット数$n$が大きくなるにつれベクトルや行列の次元は指数的に大きくなるので、愚直に計算する方法は指数的な時間を要する。

さっきの$U=H$の例だと、$\langle 0 | H | 0\rangle = 1/\sqrt{2}$であるが実際、`measure_partial`で得られる確率をみてみると、

In [9]:
simplify(measure_partial(qapply(H(1)*ctrlH*H(1)*state),(1,))[0][1])

√2   1
── + ─
4    2

となっており、上記の議論と一致していることが確認できる。

### 位相推定アルゴリズム

最終目標である位相推定アルゴリズムについて紹介する。上記のアダマールテストをより精度が良くなるように、測定側の量子ビットを拡張したのが、Kitaevによって提案された位相推定アルゴリズムである。詳細はともかく、どのような操作ができるアルゴリズムなのかを、まず紹介しよう。
　$U$を量子回路として構成できる一般的なユニタリ行列とする。$U$の固有ベクトルと$|{\rm eigen} _i \rangle$とし、対応する固有値を$|\lambda _i \rangle$とする。ある一般的な量子状態$|\psi\rangle$が与えられたとする。これは必ず固有ベクトルで展開できる：
$$
|\psi \rangle = \sum _i c_i |{\rm eigen}_i \rangle
$$
もちろん具体的にc_iがどのような値になるかはわからなくてよい。このとき位相推定アルゴリズムは、$r$個の補助量子ビットを用いて、入力状態
$$
|00...0\rangle | \psi \rangle
$$
を、
$$
\sum _i c_i |\lambda_i \rangle | {\rm eigen_i} \rangle 
$$
へと変換するアルゴリズムのことである。重ね合わせの中にあるそれぞれの固有ベクトルに対応した固有値を$r$個の補助量子ビットへと取り出すアルゴリズムになっている。この状態に対して測定をすると、確率 
$$
p_i = |c_i |^2
$$
で、どれか一つの固有ベクトルとその固有値が乱拓される。このアルゴリズムは、素因数分解や分子などのエネルギー計算のための量子化学アルゴリズム、そしてその他多くのアルゴリズムのサブルーチンとして利用されていおり、量子コンピュータが従来コンピュータよりも指数的に高速に特定の問題において解を得られる（と期待されている）最も重要な例である。

以下では、入力状態を固有状態$|{\rm eigen}\rangle$とその固有値$\lambda$に限定して、位相推定アルゴリズムを説明していくことにする。しかし、線形和をとっても全く同じ議論が使えるので一般性は失われていない。 アダマールテストでは1つしか測定用の量子ビットを使わなかった。位相推定では、測定用の補助量子ビットとして$r$個の量子ビットを確保する。
図（figure02.pdf）のように$r$個の量子ビットを用いて、
アダマールテストと同様に制御ユニタリー演算をさよう。
ただし、$k$番目（$k=0,1,...,r-1$）の補助量子ビットは制御$U^{2^k}$演算
をすることにする。
ユニタリー演算$U$の固有値の位相$\lambda$を$r$ビットの２進小数を用いて
\begin{eqnarray}
\lambda = (2\pi) 0.j_1 j_2...j_r
\end{eqnarray}
と書いておく。２進小数は$0.1=1/2、0.01=1/4、0.11=3/4$のように１未満の実数をビット列で表示したものだ。
アダマールテストと同様に
$k$番目のプローブ量子ビットには$e^{i \lambda 2^k}$
の位相が獲得される（位相キックバック）ので
\begin{eqnarray}
\bigotimes _{k=0}^{r-1} \frac{|0\rangle + e^{i (2\pi)0.j_{r-k}...j_r} |1\rangle }{\sqrt{2}}
\otimes |{\rm eigen} \rangle
\end{eqnarray}
のような状態が得られていることになる。
固有値の位相が２進数小数表示で1ビットずつシフトしたものが
各プローブ量子ビットに格納されている。
この$r$個のプローブ量子ビットに対してフーリエ変換（の逆）の量子版、逆量子フーリエ変換
\begin{eqnarray}
\bigotimes _{k=0}^{r-1} \frac{|0\rangle + e^{i (2\pi)0.j_{r-k}...j_r} |1\rangle }{\sqrt{2}}
\rightarrow |j_1...j_r\rangle
\end{eqnarray}
を作用させると、プローブ量子ビットに
ビット列 $j_1...j_r$ が得られ、位相の２進小数が得られる。
これが位相推定アルゴリズムである。逆量子フーリエ変換の構成方法については例題でみていくことにする。

SymPyで具体例を見ていこう。T演算とS演算を用いて以下のような4×4行列をつくる。

In [26]:
represent(T(0), nqubits=1)

⎡1   0  ⎤
⎢       ⎥
⎢    ⅈ⋅π⎥
⎢    ───⎥
⎢     4 ⎥
⎣0  ℯ   ⎦

In [28]:
represent(S(0), nqubits=1)

0

In [23]:
represent(T(0)*S(1),nqubits = 2)

⎡1   0    0   0  ⎤
⎢                ⎥
⎢    ⅈ⋅π         ⎥
⎢    ───         ⎥
⎢     4          ⎥
⎢0  ℯ     0   0  ⎥
⎢                ⎥
⎢0   0    1   0  ⎥
⎢                ⎥
⎢             ⅈ⋅π⎥
⎢             ───⎥
⎢              4 ⎥
⎣0   0    0  ℯ   ⎦

In [21]:
cP_2 = CGateS(2,T(0))*CGateS(2,S(1))
cP_3 = CGateS(3,T(0))*CGateS(3,S(1))
cP_4 = CGateS(4,T(0))*CGateS(4,S(1))

AttributeError: 'One' object has no attribute 'targets'